Install the Keras Tuner

1. **Install Keras Tuner:**
    - Use pip to install Keras Tuner
2. **Import necessary libraries:**
    - Import Keras Tuner, TensorFlow, and Keras modules
3. **Load and preprocess the MNIST data set:**
    - Load the MNIST data set.
    - Normalize the data set by dividing by 255.0.


In [1]:
# !pip install tensorflow==2.16.2
!pip install keras-tuner==1.4.7
# !pip install numpy<2.0.0


  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)


In [2]:
import sys
# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

In [3]:
# Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only



In [4]:
!pip install scikit-learn
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
(x_all, y_all), _ = keras.datasets.mnist.load_data()

# Flatten and normalize the images
x_all = x_all.reshape((x_all.shape[0], -1)).astype("float32") / 255.0

# Split into train+val and test (80/20)
x_temp, x_test, y_temp, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

# Split train+val into train and validation (75/25 of 80% = 60/20 overall)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

In [5]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


Defining the model with hyperparameters


**Define a model-building function:**
- Create a function `build_model` that takes a `HyperParameters` object as input.
- Use the `HyperParameters` object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.


In [11]:
# Define a model-building function

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


Configuring the hyperparameter search


**Create a RandomSearch Tuner:**
- Use the `RandomSearch` class from Keras Tuner.
- Specify the model-building function, optimization objective (validation accuracy), number of trials, and directory for storing results.


In [12]:
# Create a RandomSearch Tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()

Reloading Tuner from my_dir\intro_to_kt\tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Running the hyperparameter search



**Run the search:**
- Use the `search` method of the tuner.
- Pass in the training data, validation data, and the number of epochs


In [13]:
# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Display a summary of the results
tuner.results_summary()

Trial 10 Complete [00h 02m 53s]
val_accuracy: 0.9781999886035919

Best val_accuracy So Far: 0.9804500043392181
Total elapsed time: 00h 25m 53s
Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
units: 480
learning_rate: 0.0009507241678856559
Score: 0.9804500043392181

Trial 02 summary
Hyperparameters:
units: 256
learning_rate: 0.0012967304114198843
Score: 0.9799000024795532

Trial 09 summary
Hyperparameters:
units: 512
learning_rate: 0.0016465695698696383
Score: 0.9781999886035919

Trial 05 summary
Hyperparameters:
units: 320
learning_rate: 0.003216301914364219
Score: 0.977400004863739

Trial 07 summary
Hyperparameters:
units: 224
learning_rate: 0.00045130754049079694
Score: 0.9766499996185303

Trial 08 summary
Hyperparameters:
units: 96
learning_rate: 0.0008060771879266742
Score: 0.9736999869346619

Trial 04 summary
Hyperparameters:
units: 352
learning_rate: 0.005305163878204072
Score:

#### Explanation

This command runs the hyperparameter search:

- **`epochs=5`**: Each trial is trained for 5 epochs.
- **`validation_data=(x_val, y_val)`**: The validation data to evaluate the model's performance during the search.

After the search is complete, this command displays a summary of the best hyperparameter configurations found during the search.


 Analyzing and using the best hyperparameters



**Retrieve the best hyperparameters:**
- Use the `get_best_hyperparameters` method to get the best hyperparameters.
- Print the optimal values for the hyperparameters.

**Build and train the model:**
- Build a model using the best hyperparameters.
- Train the model on the full training data set and evaluate its performance on the test set.




In [14]:
# Retrieve the best hyperparameters

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""

The optimal number of units in the first dense layer is {best_hps.get('units')}.

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.

""")

# Build and Train the Model with Best Hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_val, y_val)
print(f'Test accuracy: {test_acc}')



The optimal number of units in the first dense layer is 480.

The optimal learning rate for the optimizer is 0.0009507241678856559.


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.8837 - loss: 0.3952 - val_accuracy: 0.9642 - val_loss: 0.1191
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9713 - loss: 0.0986 - val_accuracy: 0.9718 - val_loss: 0.0971
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9835 - loss: 0.0558 - val_accuracy: 0.9747 - val_loss: 0.0824
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9881 - loss: 0.0383 - val_accuracy: 0.9757 - val_loss: 0.0845
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9920 - loss: 0.0264 - val_accuracy: 0.9762 - val_loss: 0.0882
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9948 - loss: 0.0182 - val_accuracy: 0.9769 - val_loss: 0.0985
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9952 

#### Explanation

This code retrieves the best hyperparameters found during the search:

- **`get_best_hyperparameters(num_trials=1)`**: Gets the best hyperparameter configuration.
- **`print(f"...")`**: Prints the best hyperparameters.
- **`model.fit(...)`**: Trains the model on the full training data with a validation split of 20%.
- **`model.evaluate(...)`**: Evaluates the model on the test (validation) dataset and prints the accuracy, which gives an indication of how well the model generalizes.


 Setting Up Keras Tuner


1. Install Keras Tuner.
2. Import necessary libraries.
3. Load and preprocess the MNIST data set.


In [15]:
!pip install keras-tuner

# Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

# Load and preprocess the MNIST data set
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

# Print the shapes of the training and validation datasets
print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


Defining the model with hyperparameters

1. Define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate.
2. Compile the model with sparse categorical cross-entropy loss and Adam optimizer.


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

# Define a model-building function
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

Configuring the hyperparameter search


1. Create a `RandomSearch` tuner using the model-building function.
2. Specify the optimization objective, number of trials, and directory for storing results.


In [17]:
import keras_tuner as kt

# Create a RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,  # Ensure 'build_model' function is defined from previous code
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()

Reloading Tuner from my_dir\intro_to_kt\tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Running the hyperparameter search

1. Run the hyperparameter search using the `search` method of the tuner.
2. Pass in the training data, validation data, and the number of epochs.
3. Display a summary of the results.


In [20]:
#  Run the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Display a summary of the results
tuner.results_summary()

Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
units: 480
learning_rate: 0.0009507241678856559
Score: 0.9804500043392181

Trial 02 summary
Hyperparameters:
units: 256
learning_rate: 0.0012967304114198843
Score: 0.9799000024795532

Trial 09 summary
Hyperparameters:
units: 512
learning_rate: 0.0016465695698696383
Score: 0.9781999886035919

Trial 05 summary
Hyperparameters:
units: 320
learning_rate: 0.003216301914364219
Score: 0.977400004863739

Trial 07 summary
Hyperparameters:
units: 224
learning_rate: 0.00045130754049079694
Score: 0.9766499996185303

Trial 08 summary
Hyperparameters:
units: 96
learning_rate: 0.0008060771879266742
Score: 0.9736999869346619

Trial 04 summary
Hyperparameters:
units: 352
learning_rate: 0.005305163878204072
Score: 0.9702499806880951

Trial 06 summary
Hyperparameters:
units: 96
learning_rate: 0.00021170184837733148
Score: 0.9612999856472015

Trial 01 summ

 Analyzing and using the best hyperparameters


1. Retrieve the best hyperparameters using the `get_best_hyperparameters` method.
2. Build a model using the best hyperparameters.
3. Train the model on the full training data set and evaluate its performance on the validation set.


In [21]:
# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first dense layer is {best_hps.get('units')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

 # Build and train the model with best hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

 # Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(x_val, y_val)

print(f'Validation accuracy: {val_acc}')


The optimal number of units in the first dense layer is 480.
The optimal learning rate for the optimizer is 0.0009507241678856559.

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.8895 - loss: 0.3808 - val_accuracy: 0.9619 - val_loss: 0.1300
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.9712 - loss: 0.0964 - val_accuracy: 0.9747 - val_loss: 0.0848
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9826 - loss: 0.0571 - val_accuracy: 0.9755 - val_loss: 0.0828
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9892 - loss: 0.0363 - val_accuracy: 0.9773 - val_loss: 0.0757
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9922 - loss: 0.0274 - val_accuracy: 0.9762 - val_loss: 0.0863
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9941 - loss: 0.0190 - val_accuracy: 0.9772 - val_loss: 0.0886
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.9946 